In [28]:
import subprocess
import re
import psutil
import json
import os.path
import ipykernel
import requests
import warnings
warnings.filterwarnings('ignore')

try:  # Python 3
    from urllib.parse import urljoin
except ImportError:  # Python 2
    from urlparse import urljoin

try:  # Python 3
    from notebook.notebookapp import list_running_servers
except ImportError:  # Python 2
    import warnings
    from IPython.utils.shimmodule import ShimWarning
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=ShimWarning)
        from IPython.html.notebookapp import list_running_servers


def get_notebook_name_id_dict():
    """
    Return a dict.
        key: kernel id.
        value: the full path of the jupyter notebook.
    """
    id_name_dict = {}
    kernel_id = re.search('kernel-(.*).json',
                          ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        # replace the 'localhost' in ss['url'] with 10.0.24.31
        ss['url'] = ss['url'].replace('localhost','10.0.24.31')
        if ss.get('token', '') == '': # for jupyter not using password to auth, this won't work
            continue
        response = requests.get(urljoin(ss['url'], 'api/sessions'),
                                params={'token': ss.get('token', '')}, verify=False)
        for nn in json.loads(response.text):
            id_name_dict[nn['kernel']['id']] = os.path.join(ss['notebook_dir'], nn['notebook']['path'])
#             if nn['kernel']['id'] == kernel_id:
#                 relative_path = nn['notebook']['path']
#                 return os.path.join(ss['notebook_dir'], relative_path)
    return id_name_dict



KERNEL_REGEX = re.compile(r'.+kernel-(.+)\.json')
PID_REGEX = re.compile(r'(\d+).+')

ps = subprocess.Popen(('ps', 'ax'), stdout=subprocess.PIPE)
output = ps.communicate()[0]
import pandas as pd
results = pd.DataFrame()
id_name_dict = get_notebook_name_id_dict()
for line in output.decode().split('\n'):
    if 'kernel-' in line:
        kernel_id = re.sub(KERNEL_REGEX, r'\1', line)
        pid = int(re.sub(PID_REGEX, r'\1', line))
        process = psutil.Process(pid)
        mem = int(process.memory_info()[0])/1024/1024/1024
#         result[kernel_id]={'pid':pid,'memory': mem}
        try:
            name = id_name_dict[kernel_id]
            results.loc[name, 'process_id'] = pid
            results.loc[name, 'memory(GB)'] = mem
        except KeyError:
            print('{} not found in id_name_dict, because this kernel is running in a password protected notebook'.format(kernel_id))
try:        
    results['process_id'] = results['process_id'].astype('uint16')
except KeyError:
    print('process_id not found. results might be empty')
results

c58aa309-c3ee-4705-b8bf-d71f5946e4fc not found in id_name_dict, because this kernel is running in a password protected notebook
9ca0b38e-995c-4715-a84e-eeaac40d23d3 not found in id_name_dict, because this kernel is running in a password protected notebook
7b3d9ba0-4f82-40b5-acf0-56c25e8c8a6b not found in id_name_dict, because this kernel is running in a password protected notebook
a553b672-8f78-45a1-a36c-2fb131c94f88 not found in id_name_dict, because this kernel is running in a password protected notebook
0e471a71-e71c-4b4a-ae4c-c37d942c1687 not found in id_name_dict, because this kernel is running in a password protected notebook
24d05b1e-340a-4d9b-b817-3ff4be895bca not found in id_name_dict, because this kernel is running in a password protected notebook
43476104-e1d8-4125-b8f8-b20d27501138 not found in id_name_dict, because this kernel is running in a password protected notebook
3981901f-a229-42d3-95e7-ecaeac60ae0b not found in id_name_dict, because this kernel is running in a pass

,process_id,memory(GB)
/home/kai/data/kernel_memory_info.ipynb,1056,0.099594


# maybe useful in future

In [ ]:
from tornado import gen, web
from notebook.base.handlers import APIHandler

class KernelMemoryHandler(APIHandler):
    '''
    get memory of each kernel
    '''
    @web.authenticated
    @gen.coroutine

    def get(self):
        ps = subprocess.Popen(('ps', 'ax'), stdout=subprocess.PIPE)
        output = ps.communicate()[0]
        result={}
        for line in output.split('\n'):
            if 'kernel-' in line:
                kernel_id = re.sub(KERNEL_REGEX, r'\1', line)
                pid = int(re.sub(PID_REGEX, r'\1', line))
                process = psutil.Process(pid)
                mem = int(process.memory_info()[0])
                print(kernel_id, pid, process, mem)
                result[kernel_id]={'pid':pid,'memory': mem}
        self.finish(result)
       
default_handlers = [
    (r"/api/kernel/memory", KernelMemoryHandler)
]